In [1]:
import pandas as pd
from datetime import datetime
from neo4j import GraphDatabase



def load_data():
    df = pd.read_csv(
        'US.60826002.csv',
        header=0,
        names=['state', 'period_start', 'period_end', 'cumulative', 'count'],
        usecols=[8, 11, 12, 13, 19],
    )
    df.replace({r'^US-': ''}, regex=True, inplace=True)
    return df

def load_states():
    df = pd.read_csv(
        'US.60826002.csv',
        header=0,
        names=['statename', 'state'],
        usecols=[8, 7],
    )
    df.replace({r'^US-': ''}, regex=True, inplace=True)
    return df

def load_periods():
    df = pd.read_csv(
        'US.60826002.csv',
        header=0,
        names=['period_start', 'period_end'],
        usecols=[11, 12],
    )
    df.replace({r'^US-': ''}, regex=True, inplace=True)
    
    df = df.drop_duplicates()
    
    period_length = []

    for ind in df.index:
         period_length.append((datetime.strptime(df['period_end'][ind], '%Y-%m-%d') - datetime.strptime(df['period_start'][ind], '%Y-%m-%d')).days)
    df['period_length'] = period_length
    
    return df

In [2]:
df = load_data()
df2 = load_states()
df3 = load_periods()

In [3]:
print(df)

      state period_start  period_end  cumulative  count
0        WI   2007-02-18  2007-02-24           0      0
1        OH   2006-02-12  2006-02-18           0      1
2        OH   2006-03-05  2006-03-11           0      1
3        OH   2006-03-12  2006-03-18           0      1
4        OH   2006-04-16  2006-04-22           0      1
...     ...          ...         ...         ...    ...
10673    PA   2012-12-30  2013-12-07           1      1
10674    PA   2012-12-30  2013-12-14           1      1
10675    PA   2012-12-30  2013-12-21           1      1
10676    PA   2012-12-30  2013-12-28           1      1
10677    VI   2006-01-01  2006-06-03           1      0

[10678 rows x 5 columns]


In [4]:
df2 = df2.drop_duplicates()

print(df2)

                 statename state
0                WISCONSIN    WI
1                     OHIO    OH
129               MICHIGAN    MI
174                 NEVADA    NV
323               KENTUCKY    KY
324                WYOMING    WY
334          NEW HAMPSHIRE    NH
336            MISSISSIPPI    MS
337               NEBRASKA    NE
366                ARIZONA    AZ
865              MINNESOTA    MN
881               ARKANSAS    AR
885           NORTH DAKOTA    ND
887               MARYLAND    MD
920         SOUTH CAROLINA    SC
921           RHODE ISLAND    RI
922                MONTANA    MT
936                   UTAH    UT
948               MISSOURI    MO
970             CALIFORNIA    CA
1461                OREGON    OR
1468             LOUISIANA    LA
1474          PENNSYLVANIA    PA
1475  VIRGIN ISLANDS, U.S.    VI
3683              DELAWARE    DE
4667              ILLINOIS    IL
6414  DISTRICT OF COLUMBIA    DC
6984            NEW MEXICO    NM
7578                  IOWA    IA
7655      

In [5]:
print(df3)

     period_start  period_end  period_length
0      2007-02-18  2007-02-24              6
1      2006-02-12  2006-02-18              6
2      2006-03-05  2006-03-11              6
3      2006-03-12  2006-03-18              6
4      2006-04-16  2006-04-22              6
...           ...         ...            ...
8072   2004-01-04  2004-01-31             27
8073   2004-01-04  2004-02-07             34
9198   2004-01-04  2004-01-10              6
9199   2004-01-04  2004-01-17             13
9300   2006-01-01  2006-01-14             13

[1321 rows x 3 columns]


In [10]:
####### import states into neo4j ##### 
dataframe_list = df2.values.tolist()

dataframe_execution_commands = []

for i in dataframe_list:
    neo4j_create_statemenet = "create (s:State {state:" + "\"" + str(i[1]) +"\", statename:  " + "\"" + str(i[0]) +"\"" + "})"

    dataframe_execution_commands.append(neo4j_create_statemenet)

    
def execute_transactions(dataframe_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "123")) # 123 = password
    session = data_base_connection.session()    
    for i in dataframe_execution_commands:
        session.run(i)

        
execute_transactions(dataframe_execution_commands)



#print(dataframe_execution_commands)

In [7]:
####### import time_periods into neo4j ##### 
dataframe_list = df3.values.tolist()

dataframe_execution_commands = []

for i in dataframe_list:
    neo4j_create_statemenet = "create (t:Timeperiod {period_start:" + "\"" + str(i[0]) +"\", period_end:" + "\"" + str(i[1]) +"\", period_length:" + str(i[2]) + "})"

    dataframe_execution_commands.append(neo4j_create_statemenet)

    
def execute_transactions(dataframe_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "123")) # 123 = password
    session = data_base_connection.session()    
    for i in dataframe_execution_commands:
        session.run(i)

        
execute_transactions(dataframe_execution_commands)

In [8]:
####### import data into neo4j ##### 

dataframe_list = df.values.tolist()

dataframe_execution_commands = []

for i in dataframe_list:
    neo4j_create_statemenet = "create (r:Record {state:" + "\"" + str(i[0]) +"\", period_start:  " + "\"" + str(i[1]) +"\", period_end: " + "\""  + str(i[2]) +"\", cumulative: " + str(i[3]) + ", count: "+ str(i[4]) + "})"

    dataframe_execution_commands.append(neo4j_create_statemenet)

    
def execute_transactions(dataframe_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "123")) # 123 = password
    session = data_base_connection.session()    
    for i in dataframe_execution_commands:
        session.run(i)


execute_transactions(dataframe_execution_commands)

#print(dataframe_execution_commands)

In [9]:
###### create relationships #####

#relationship from record to state
execute_transactions(["MATCH (a:Record), (b:State) WHERE a.state = b.state CREATE (a)-[r:IsInState]->(b)"])

#relationship from record to time_period
execute_transactions(["MATCH (a:Record), (b:Timeperiod) WHERE a.period_start = b.period_start AND a.period_end = b.period_end  CREATE (a)-[r:IsInPeriod]->(b)"])

# Queries ran in neo4j:


1. graph of all records and what state they belong to: https://i.imgur.com/bMz5jhE.png

```
MATCH p=()-[r:IsInState]->() 
RETURN p LIMIT 500
```

2. graph of all records and what timeperiod they belong to: https://i.imgur.com/YVh5Y5E.png

```
MATCH p=()-[r:IsInPeriod]->() 
RETURN p LIMIT 1000
```

3. State and max number of cases in a timeperiod: https://i.imgur.com/vZYs8lu.png

```
MATCH (n:Record) -   [r] - (m:State)
RETURN m.statename, max(n.count) , max(n.period_start), max(n.period_end)
ORDER BY max(n.count) DESC
```

4. records with more than 500 infected, and what state it was in: https://i.imgur.com/tCySXe2.png

```
MATCH (n:Record) -   [r] - (m:State)
where n.count > 500
RETURN n,r,m
```

5. max infected in a 1 week period for each state: https://i.imgur.com/28Hqbqe.png

```
MATCH (n:Record) -   [r] - (m)
WHERE m.period_length = 6
RETURN n.state, max(n.count)
ORDER BY max(n.count) DESC
```

6. max daily infection rates for each state: https://i.imgur.com/EYAadcU.png

```
MATCH (n:Record) -   [r] - (m)
RETURN n.state, max(n.count/m.period_length)
ORDER BY max(n.count/m.period_length) DESC
```